# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("cities.csv")

weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kieta,90,PG,1601559593,81,-6.22,155.63,77.65,1.01
1,1,Yangi Marg`ilon,66,UZ,1601559624,55,40.43,71.72,62.60,2.24
2,2,Ushuaia,75,AR,1601559625,100,-54.80,-68.30,26.60,14.99
3,3,Cape Town,40,ZA,1601559418,51,-33.93,18.42,62.60,31.09
4,4,Touros,75,BR,1601559625,83,-5.20,-35.46,77.00,10.29
...,...,...,...,...,...,...,...,...,...,...
547,547,Celestún,95,MX,1601559849,88,20.87,-90.40,78.80,14.29
548,548,Umluj,0,SA,1601559850,56,25.02,37.27,89.58,5.35
549,549,Cabo Rojo,57,PR,1601559850,68,18.09,-67.15,89.01,1.99
550,550,São Raimundo Nonato,1,BR,1601559850,30,-9.02,-42.70,91.67,11.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]


In [4]:


# Create  Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
vacation_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 8) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,Guerrero Negro,0,MX,1601559627,54,27.98,-114.06,73.90,1.77
53,53,Port Hedland,0,AU,1601559645,36,-20.32,118.57,75.20,6.93
133,133,Mount Isa,0,AU,1601559679,22,-20.73,139.50,77.00,6.93
231,231,Tomatlán,0,MX,1601559722,85,19.93,-105.25,73.69,3.33
245,245,Aiquile,0,BO,1601559729,34,-18.20,-65.18,73.80,6.06
308,308,Rodolfo Sánchez Taboada,0,MX,1601559752,30,31.72,-116.57,73.94,2.48
309,309,Alice Springs,0,AU,1601559598,13,-23.70,133.88,75.20,6.93
345,345,Fomboni,0,KM,1601559768,75,-12.28,43.74,79.81,7.63
371,371,San Fernando del Valle de Catamarca,0,AR,1601559642,39,-28.47,-65.79,73.99,1.99
482,482,San Quintín,0,MX,1601559823,42,30.48,-115.95,75.74,4.32


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Create a hotel_df
hotel_df = vacation_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Guerrero Negro,MX,27.98,-114.06,
53,Port Hedland,AU,-20.32,118.57,
133,Mount Isa,AU,-20.73,139.50,
231,Tomatlán,MX,19.93,-105.25,
245,Aiquile,BO,-18.20,-65.18,
308,Rodolfo Sánchez Taboada,MX,31.72,-116.57,
309,Alice Springs,AU,-23.70,133.88,
345,Fomboni,KM,-12.28,43.74,
371,San Fernando del Valle de Catamarca,AR,-28.47,-65.79,
482,San Quintín,MX,30.48,-115.95,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [15]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # build url and make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
     # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 9: Guerrero Negro.
Closest hotel in Guerrero Negro is The Halfway Inn.
Retrieving Results for Index 53: Port Hedland.
Closest hotel in Port Hedland is The Esplanade Hotel.
Retrieving Results for Index 133: Mount Isa.
Closest hotel in Mount Isa is ibis Styles Mt Isa Verona.
Retrieving Results for Index 231: Tomatlán.
Closest hotel in Tomatlán is Hotel Misión San Miguel.
Retrieving Results for Index 245: Aiquile.
Closest hotel in Aiquile is Isola Hotel.
Retrieving Results for Index 308: Rodolfo Sánchez Taboada.
Missing field/result... skipping.
Retrieving Results for Index 309: Alice Springs.
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
Retrieving Results for Index 345: Fomboni.
Closest hotel in Fomboni is Le Relais de Singani hôtel.
Retrieving Results for Index 371: San Fernando del Valle de Catamarca.
Closest hotel in San Fernando del Valle de Catamarca is Hotel Casino Catamarca.
Retrieving Results for Index 482: San Quintín.


In [16]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Guerrero Negro,MX,27.98,-114.06,The Halfway Inn
53,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
133,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
231,Tomatlán,MX,19.93,-105.25,Hotel Misión San Miguel
245,Aiquile,BO,-18.20,-65.18,Isola Hotel
308,Rodolfo Sánchez Taboada,MX,31.72,-116.57,
309,Alice Springs,AU,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
345,Fomboni,KM,-12.28,43.74,Le Relais de Singani hôtel
371,San Fernando del Valle de Catamarca,AR,-28.47,-65.79,Hotel Casino Catamarca
482,San Quintín,MX,30.48,-115.95,Old Mill Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info =[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
 

# Add marker layer and info box content on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig



Figure(layout=FigureLayout(height='420px'))